<a href="https://colab.research.google.com/github/Nithin46/Project_Dummy/blob/main/IMDB_Enhance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
Net = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/netflix_titles.csv', low_memory=False)
Net.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,14-Aug-20,2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,23-Dec-16,2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,20-Dec-18,2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."


In [3]:
IMDB = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/IMDb movies.csv',low_memory=False)
IMDB.head(3)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0


In [4]:
new_ratings = pd.DataFrame({'Title':IMDB.title,
                    'Rating': IMDB.avg_vote})

new_ratings.drop_duplicates(subset=['Title', 'Rating'], inplace=True)

print(new_ratings.shape)
new_ratings.head(5)

(85733, 2)


,Title,Rating
0,Miss Jerry,5.9
1,The Story of the Kelly Gang,6.1
2,Den sorte drøm,5.8
3,Cleopatra,5.2
4,L'Inferno,7.0


In [5]:
Inner_join_data = new_ratings.merge(Net,left_on='Title', right_on='title', how='inner')
Inner_join_data=Inner_join_data.sort_values(by='Rating', ascending=False)

print(Inner_join_data.shape)
Inner_join_data.head(5)

(2721, 14)


,Title,Rating,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
977,Innocent,9.0,s3009,TV Show,Innocent,Seren Yüce,"Ali Atay, Haluk Bilginer, Nur Sürer, Okan Yala...",Turkey,23-Jan-19,2017,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Dramas","In a peaceful, rustic town, a retired officer ..."
988,Breakout,9.0,s1093,TV Show,Breakout,NaN,"Jeanette Aw, Elvin Ng, Zhou Ying, Christopher ...",NaN,1-Jul-17,2010,TV-14,1 Season,"International TV Shows, TV Dramas",After losing her family in a suspicious car cr...
389,Pulp Fiction,8.9,s5003,Movie,Pulp Fiction,Quentin Tarantino,"John Travolta, Samuel L. Jackson, Uma Thurman,...",United States,1-Jan-19,1994,R,154 min,"Classic Movies, Cult Movies, Dramas",This stylized crime caper weaves together stor...
371,Schindler's List,8.9,s5431,Movie,Schindler's List,Steven Spielberg,"Liam Neeson, Ben Kingsley, Ralph Fiennes, Caro...",United States,1-Apr-18,1993,R,195 min,"Classic Movies, Dramas",Oskar Schindler becomes an unlikely humanitari...
1285,Inception,8.8,s2980,Movie,Inception,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen...","United States, United Kingdom",1-Jan-20,2010,PG-13,148 min,"Action & Adventure, Sci-Fi & Fantasy, Thrillers","In this mind-bending sci-fi thriller, a man ru..."


In [6]:
New_Data = Inner_join_data[['Title', 'Rating', 'type']]

New_Data.drop_duplicates(subset=['Title','Rating', 'type'], inplace=True)
print(New_Data.shape)
New_Data.head(5)

(2721, 3)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Title,Rating,type
977,Innocent,9.0,TV Show
988,Breakout,9.0,TV Show
389,Pulp Fiction,8.9,Movie
371,Schindler's List,8.9,Movie
1285,Inception,8.8,Movie


In [7]:
Movies_Data = New_Data[New_Data.type == 'Movie']
TV_Data = New_Data[New_Data.type == 'TV Show']
print(Movies_Data.shape)
TV_Data.shape

(2384, 3)


(337, 3)

In [8]:
Movies_Data = Movies_Data.drop(['type'], axis=1)

In [9]:
Movies_Data

,Title,Rating
389,Pulp Fiction,8.9
371,Schindler's List,8.9
1285,Inception,8.8
2651,Ani... Dr. Kashinath Ghanekar,8.6
598,Koshish,8.6
...,...,...
832,The Vault,1.9
571,Himmatwala,1.7
2175,Aerials,1.6
1189,Pink,1.6


In [10]:
Movies_Data['Polarity_Rating'] = Movies_Data['Rating'].apply(lambda x: 'Positive' if x > 6 else 'Negative')
Movies_Data

,Title,Rating,Polarity_Rating
389,Pulp Fiction,8.9,Positive
371,Schindler's List,8.9,Positive
1285,Inception,8.8,Positive
2651,Ani... Dr. Kashinath Ghanekar,8.6,Positive
598,Koshish,8.6,Positive
...,...,...,...
832,The Vault,1.9,Negative
571,Himmatwala,1.7,Negative
2175,Aerials,1.6,Negative
1189,Pink,1.6,Negative


In [11]:
fig = px.pie(values=Movies_Data['Polarity_Rating'].value_counts(), 
             names=Movies_Data['Polarity_Rating'].value_counts().index)
fig.show()

In [12]:
Positive = Movies_Data[Movies_Data['Polarity_Rating'] == 'Positive']
Negative = Movies_Data[Movies_Data['Polarity_Rating'] == 'Negative']

print(Positive.shape)
Negative.shape

(1206, 3)


(1178, 3)

In [13]:
df = Movies_Data[['Title','Polarity_Rating']]
df

,Title,Polarity_Rating
389,Pulp Fiction,Positive
371,Schindler's List,Positive
1285,Inception,Positive
2651,Ani... Dr. Kashinath Ghanekar,Positive
598,Koshish,Positive
...,...,...
832,The Vault,Negative
571,Himmatwala,Negative
2175,Aerials,Negative
1189,Pink,Negative


In [14]:
one_hot = pd.get_dummies(df["Polarity_Rating"])
df.drop(['Polarity_Rating'],axis=1,inplace=True)
df = pd.concat([df,one_hot],axis=1)
df

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Title,Negative,Positive
389,Pulp Fiction,0,1
371,Schindler's List,0,1
1285,Inception,0,1
2651,Ani... Dr. Kashinath Ghanekar,0,1
598,Koshish,0,1
...,...,...,...
832,The Vault,1,0
571,Himmatwala,1,0
2175,Aerials,1,0
1189,Pink,1,0


In [15]:
X = df['Title'].values
y = df.drop('Title', axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [16]:
X_train

array(['Anchor and Hope', 'Kya Kehna', 'VS.', ..., 'Leap Year',
       'The Breaker Upperers', 'The Rehearsal'], dtype=object)

In [17]:
y_train

array([[0, 1],
       [1, 0],
       [1, 0],
       ...,
       [0, 1],
       [1, 0],
       [0, 1]], dtype=uint8)

In [18]:
vect = CountVectorizer()
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

In [19]:
tfidf = TfidfTransformer()
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)
X_train = X_train.toarray()
X_test = X_test.toarray()

In [20]:
model = Sequential()

model.add(Dense(units=12673,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=4000,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=500,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=2, activation='sigmoid'))

opt=tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)

In [21]:
model.fit(x=X_train, y=y_train, batch_size=100, epochs=50, validation_data=(X_test, y_test), verbose=1, callbacks=early_stop)

Epoch 1/50
22/22 [==============================] - 3s 99ms/step - loss: 0.7028 - accuracy: 0.4935 - val_loss: 0.6825 - val_accuracy: 0.5732
Epoch 2/50
22/22 [==============================] - 1s 66ms/step - loss: 0.4824 - accuracy: 0.8616 - val_loss: 0.8491 - val_accuracy: 0.5481
Epoch 3/50
22/22 [==============================] - 1s 66ms/step - loss: 0.2254 - accuracy: 0.9119 - val_loss: 0.8811 - val_accuracy: 0.5983
Epoch 4/50
22/22 [==============================] - 1s 65ms/step - loss: 0.1648 - accuracy: 0.9181 - val_loss: 0.9695 - val_accuracy: 0.5774
Epoch 5/50
22/22 [==============================] - 1s 67ms/step - loss: 0.1425 - accuracy: 0.9171 - val_loss: 1.1013 - val_accuracy: 0.5607
Epoch 00005: early stopping


In [22]:
model_score = model.evaluate(X_test, y_test, batch_size=64, verbose=1)
print('Test accuracy:', model_score[1])

4/4 [==============================] - 0s 11ms/step - loss: 1.1013 - accuracy: 0.5607
Test accuracy: 0.560669481754303


In [23]:
preds = model.predict(X_test)
preds

array([[0.612349  , 0.4297736 ],
       [0.9516913 , 0.13266265],
       [0.53155893, 0.48956534],
       [0.45316383, 0.5525985 ],
       [0.5088444 , 0.5067268 ],
       [0.5793237 , 0.45383412],
       [0.55439496, 0.4650781 ],
       [0.98941386, 0.05467955],
       [0.07311551, 0.9345067 ],
       [0.4814024 , 0.52648807],
       [0.9474665 , 0.1392333 ],
       [0.55439496, 0.4650781 ],
       [0.9799033 , 0.07913845],
       [0.52699864, 0.4961787 ],
       [0.55439496, 0.4650781 ],
       [0.43438452, 0.57018065],
       [0.28018266, 0.7197895 ],
       [0.55439496, 0.4650781 ],
       [0.4691113 , 0.5379197 ],
       [0.55439496, 0.4650781 ],
       [0.09142755, 0.91636914],
       [0.9188905 , 0.17938527],
       [0.78312355, 0.31149566],
       [0.55439496, 0.4650781 ],
       [0.99752   , 0.02239846],
       [0.99489474, 0.03455364],
       [0.5639634 , 0.4654439 ],
       [0.5271595 , 0.49476388],
       [0.55439496, 0.4650781 ],
       [0.5753001 , 0.46061176],
       [0.